# Banca d'Italia

In [1]:
import requests, pandas as pd, os, sqlalchemy, sqlite3, sys, duckdb
from io import BytesIO
from sqlalchemy import create_engine

sqlite  = create_engine('sqlite:///D:/Bankit.sqlite')
ddb     = duckdb.connect('D:/Bankit.duckdb')

In [6]:
ddb.close()

In [8]:
# Banche e moneta: serie nazionali #BAM mese https://www.bancaditalia.it/pubblicazioni/moneta-banche/index.html
STAMEN =['AGGM0100','AGGM0200','AGGM0300','AGGM0400','AGGM0500','ATECO100','BMON0100','BSIB0100','BSIB0200','BSIB0300','BSIB0400','BSIB0500','BSIB0600','BSIB0700', 'BSIB0800','BSIB1000','BSIB1010','BSIB1100','BSIB1110','BSID0100','BSID0200','BSIO0100',
         'BSIO0200','CARB0100','CARB0200','CARB0300','CE00100','MID0100','MIR0100','MIR0200','MIR0300','MIR0400','MIR0500','MIR0600','MIR0700','MIR0800','MIR0900','ROB0100','SPBI0100','SPBI0200','TITD0100','TITD0200','TITP0100'] # fare check di 'OPM0100','TUFF0100'
# Banche e istituzioni finanziarie: finanziamenti e raccolta per settori e territori  fine trimestre https://www.bancaditalia.it/pubblicazioni/finanziamenti-raccolta/
STAFINRA = ['TDB10290','TDB10221','TDB10224','TDB10226','TDB10295','TDB20224','TDB20226','TDB20290','TDB20295', 'TFR40020','TFR20232','TFR20255','TFR20231','TFR30274','TFR10425','TFR10435','TFR20281','TFR10254','TFR30309','TFR30315','TFR20163',
    'TFR20269','TFR20267','TFR40082','TFR40087','TFR40500','TFR40300','TFR40400','TFR30980','TFR30970','TFR10194','TFR10286','TFR10241','TFR10232','TFR10255','TFR10281','TFR10236','TFR10420','TFR10430','TFR10460','TFR10194','TFR10283','TFR10287',
    'TFR10163','TFR10269','TFR10267','TFR40100','TFR10288','TFR10289','TFR20236']

# Banche e istituzioni finanziarie: CONDIZIONI e RISCHIOSITA' del credito per settori e territori fine trimestre https://www.bancaditalia.it/pubblicazioni/condizioni-rischiosita/
STACORIS = ['TRI30266','TRI30267','TRI30265','TRI30271','TRI30601','TRI30602','TRI30603','TRI30604','TRI30605','TRI30606','TRI30486','TRI30496','TRI30507','TRI30516','TRI30524','TRI30529',
            'TRI30631','TRI30632','TRI30634','TRI30635','TRI30636','TRI30431','TRI30446','TRI30466','TRI30476','TRI30361','TRI30401','TRI30101','TRI30126','TRI30146','TRI30156',
            'TRI30190','TRI30136','TRI30166','TRI30871','TRI30881','TRI30890','TRI30900','TRI30950','TRI30951','TRI30952','TRI31100','TRI31101','TRI30021','TRI30171','TRI30181','TRI30268',
            'TRI30269','TRI30206','TRI30031','TRI30231','TRI30226','TRI30033','TRI30211','TRI30241','TRI30251','TRI30290','TRI30953','TRI30954','TRI30955','TRI31102','TRI31103']
# 'TRI30633',
AdHoc = ['TRI30634','TRI30635','TRI30636','TRI30431','TRI30446','TRI30466','TRI30476','TRI30361','TRI30401','TRI30101','TRI30126','TRI30146','TRI30156',
            'TRI30190','TRI30136','TRI30166','TRI30871','TRI30881','TRI30890','TRI30900','TRI30950','TRI30951','TRI30952','TRI31100','TRI31101','TRI30021','TRI30171','TRI30181','TRI30268',
            'TRI30269','TRI30206','TRI30031','TRI30231','TRI30226','TRI30033','TRI30211','TRI30241','TRI30251','TRI30290','TRI30953','TRI30954','TRI30955','TRI31102','TRI31103']

STAATER = ['TDB20207','TDB20212','TDB20220','TDB10222','TDB10225','TDB10227','TDB20210','TDB20225','TDB20230','TDB10195']
tabelle = ['TDB20207','TDB10219','TDB10224','TDB10226','TDB10232','TDB10295','TDB20212','TDB20220','TDB20224','TDB20290','TDB20295','TFR10194','TFR10232', 'TFR10241','TFR10255','TFR10420','TFR10425','TFR10430','TFR20231','TFR20232','TFR20236','TFR20255','TFR20267',
           'TFR20281','TFR30309','TFR30315','TFR40100','TRI30021','TRI30171','TRI30211','TRI30431','TRI30446']
singola = ['TFR40020']
vecchie = ['TDB10194','TDB10255']

## inquiry su tabelle/viste esistenti

In [77]:
ddb.execute(f"SELECT * from V_TDB10224_stafinra where LOC_CTP = 'ITG2E' order by DATA_OSS desc LIMIT 5").df()


,DATA_OSS,ENTE_SEGN,SEGNALANTE,LOC_CTP,AREA,SET_CTP,TARGET,FENEC,FENOMENO,ATECO_CTP,ATECO,VALORE
0,2024-12-31,1070001,Banche e Cassa depositi e prestiti,ITG2E,None,SBI42,Totale residenti al netto delle Istituzioni fi...,52000139,None,1005009,Totale ateco al netto della sez. U comprese le...,2211368.0
1,2024-12-31,1070001,Banche e Cassa depositi e prestiti,ITG2E,None,SBI25,Società non finanziarie e famiglie produttrici,52000139,None,1005003,Servizi,567228.0
2,2024-12-31,1070001,Banche e Cassa depositi e prestiti,ITG2E,None,SBI25,Società non finanziarie e famiglie produttrici,52000139,None,F,Costruzioni,67010.0
3,2024-12-31,1070001,Banche e Cassa depositi e prestiti,ITG2E,None,SBI25,Società non finanziarie e famiglie produttrici,52000139,None,1004999,Totale ateco al netto della sez. U,938598.0
4,2024-12-31,1070001,Banche e Cassa depositi e prestiti,ITG2E,None,SBI25,Società non finanziarie e famiglie produttrici,52000139,None,1005001,Attività industriali,161246.0


In [69]:
ddb.execute(f"SELECT LOC_CTP, AREA from V_TDB10224_stafinra where LOC_CTP like 'ITG2%' group by 1,2").df()


,LOC_CTP,AREA
0,ITG2G,None
1,ITG28,Oristano
2,ITG2H,Sud Sardegna
3,ITG2E,None
4,ITG2,Sardegna
5,ITG25,Sassari
6,ITG2D,None
7,ITG2B,Medio Campidano
8,ITG29,Olbia-Tempio
9,ITG2C,Carbonia-Iglesias


## aggiornamento tabella

In [78]:
query_create = f""" UPDATE stafinra
SET Descrizione = CASE Elemento
    WHEN 'ITC2F' THEN 'Cagliari'
    ELSE Descrizione
END
WHERE Elemento IN ('ITC2F');"""

## creazione una tantum di viste

In [4]:
query_create = f""" UPDATE stafinra
SET Descrizione = CASE Elemento
    WHEN 'ITC20' THEN 'Liguria'
    WHEN 'ITG2D' THEN 'Sardegna Sud'
    WHEN 'ITG2E' THEN 'Sardegna Cagliari'
    WHEN 'ITG2F' THEN 'Sardegna Nord'
    WHEN 'ITG2G' THEN 'Sardegna Est'
    ELSE Descrizione
END
WHERE Elemento IN ('ITC20', 'ITG2D', 'ITG2E', 'ITG2F', 'ITG2G');"""

In [40]:
# script valido per 'TDB10224','TFR20232'
tabella = 'TDB10224'
dominio = 'stafinra'
vista = f"V_{tabella}_{dominio}"  # Qui usiamo f-string per creare il nome dinamico della vista
query_drop = f"DROP VIEW IF EXISTS {vista};"
ddb.execute(query_drop)
query_create = f""" CREATE VIEW {vista} AS SELECT  d.DATA_OSS,
        d.ENTE_SEGN, s1.Descrizione AS SEGNALANTE,
        d.LOC_CTP,   s2.Descrizione AS AREA,
        d.SET_CTP,   s3.Descrizione AS TARGET,
        d.FENEC,     s4.Descrizione AS FENOMENO,
        d.ATECO_CTP, s5.Descrizione AS ATECO,
        d.VALORE
FROM {tabella} d
LEFT JOIN {dominio} s1 ON d.ENTE_SEGN = s1.Elemento
LEFT JOIN {dominio} s2 ON d.LOC_CTP = s2.Elemento
LEFT JOIN {dominio} s3 ON d.SET_CTP = s3.Elemento
LEFT JOIN {dominio} s4 ON d.FENEC = s4.Elemento
LEFT JOIN {dominio} s5 ON d.ATECO_CTP = s5.Elemento; """



In [42]:
ddb.execute(query_create)
df = ddb.execute(f"SELECT * FROM {vista} limit 5").fetchdf()
df

CatalogException: Catalog Error: View with name "V_TDB10224_stafinra" already exists!

In [19]:
# script valido per 'TDB10226','TDB10295'
tabella = 'TFR20232'
dominio = 'stafinra'
vista = f"V_{tabella}_{dominio}"  # Qui usiamo f-string per creare il nome dinamico della vista
query_drop = f"DROP VIEW IF EXISTS {vista};"
ddb.execute(query_drop)
query_create = f""" CREATE VIEW {vista} AS SELECT  d.DATA_OSS,
        d.ENTE_SEGN, s1.Descrizione AS SEGNALANTE,
        d.LOC_CTP,   s2.Descrizione AS AREA,
        d.SET_CTP,   s3.Descrizione AS TARGET,
        d.FENEC,     s4.Descrizione AS FENOMENO,
        d.VALORE
FROM {tabella} d
LEFT JOIN {dominio} s1 ON d.ENTE_SEGN = s1.Elemento
LEFT JOIN {dominio} s2 ON d.LOC_CTP = s2.Elemento
LEFT JOIN {dominio} s3 ON d.SET_CTP = s3.Elemento
LEFT JOIN {dominio} s4 ON d.FENEC = s4.Elemento; """

In [21]:
# script valido per 'TDB10224','TDB10226','TDB10295'
tabella = 'TFR20232'
dominio = 'stafinra'
vista = f"V_{tabella}_{dominio}"  # Qui usiamo f-string per creare il nome dinamico della vista
query_drop = f"DROP VIEW IF EXISTS {vista};"
ddb.execute(query_drop)
query_create = f"""CREATE VIEW {vista} AS SELECT d.DATA_OSS,
    d.ENTE_SEGN,    s1.Descrizione AS SEGNALANTE,
    d.LOC_SPORT,    s2.Descrizione AS AREA,
    d.FENEC,        s3.Descrizione AS FENOMENO,
    d.VALORE
FROM {tabella} d
LEFT JOIN {dominio} s1 ON d.ENTE_SEGN = s1.Elemento
LEFT JOIN {dominio} s2 ON d.LOC_SPORT = s2.Elemento
LEFT JOIN {dominio} s3 ON d.FENEC     = s3.Elemento;"""


In [ ]:
# script valido per 'TRI30156',
tabella = 'TRI30156'
dominio = 'stacoris'
vista = f"V_{tabella}_{dominio}"  # Qui usiamo f-string per creare il nome dinamico della vista
query_drop = f"DROP VIEW IF EXISTS {vista};"
ddb.execute(query_drop)

query_create = f"""CREATE VIEW {vista} AS SELECT d.DATA_OSS,
    d.ENTE_SEGN,     s1.Descrizione AS SEGNALANTE,
    d.SET_CTP,     s2.Descrizione AS AREA,
    d.FENEC,     s3.Descrizione AS FENOMENO,
    d.ATECO_CTP,     s4.Descrizione AS SETTORE_ATECO,
    d.CLASSE_ACCORD,     s5.Descrizione AS CLASSE_ACCORDO,
    d.SEDELEG_SOGG,     s6.Descrizione AS SEDE_LEGALE,
    d.VALORE
FROM {tabella} d
LEFT JOIN {dominio} s1 ON d.ENTE_SEGN = s1.Elemento
LEFT JOIN {dominio} s2 ON d.SET_CTP   = s2.Elemento
LEFT JOIN {dominio} s3 ON d.FENEC     = s3.Elemento
LEFT JOIN {dominio} s4 ON d.ATECO_CTP = s4.Elemento
LEFT JOIN {dominio} s5 ON CAST(d.CLASSE_ACCORD AS VARCHAR) = s5.Elemento
LEFT JOIN {dominio} s6 ON d.SEDELEG_SOGG = s6.Elemento; """
ddb.execute(query_create)
print(f"✅ Vista '{vista}' ricreata con successo!")

✅ Vista 'V_TRI30156_stacoris' ricreata con successo!


In [32]:
# vedere le viste memorizzate
query_create = f"""SELECT table_name, table_type 
FROM information_schema.tables 
WHERE table_schema = 'main' AND table_type = 'VIEW';"""
ddb.execute(query_create).fetchdf() 

,table_name,table_type
0,V_TDB10224_stafinra,VIEW
1,V_TDB10226_stafinra,VIEW
2,V_TDB10295_stafinra,VIEW
3,V_TFR10194_stafinra,VIEW
4,V_TFR10255_stafinra,VIEW
5,V_TFR20232_stafinra,VIEW
6,V_TRI30156_stacoris,VIEW


In [34]:
# vedere le colonne
query_create = f"""DESCRIBE {vista};"""
ddb.execute(query_create).fetchdf() 

,column_name,column_type,null,key,default,extra
0,DATA_OSS,TIMESTAMP_NS,YES,None,None,None
1,ENTE_SEGN,VARCHAR,YES,None,None,None
2,SEGNALANTE,VARCHAR,YES,None,None,None
3,LOC_CTP,VARCHAR,YES,None,None,None
4,AREA,VARCHAR,YES,None,None,None
5,SET_CTP,VARCHAR,YES,None,None,None
6,TARGET,VARCHAR,YES,None,None,None
7,FENEC,VARCHAR,YES,None,None,None
8,FENOMENO,VARCHAR,YES,None,None,None
9,ATECO_CTP,VARCHAR,YES,None,None,None


## **aggiornamento periodico tabelle**

|set|20250331|
|-|-|
|STAATER|x|
|STAFINRA||
|STACORIS||

In [3]:
def carica_dati_in_sql(tabella):
    file_url = f"https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}"
    result = requests.get(file_url)
    date_column = ['DATA_OSS']
    df = pd.read_csv(BytesIO(result.content),
                 compression='zip',
                 header=0,
                 sep=';',
                 quotechar='"',
                 encoding='utf-8',
                 decimal=',',  # Gestisce numeri come "-5,8"
                 dtype={'ENTE_SEGN': 'str', 'FENEC': 'str', 'VALORE': 'float64', 'LOC_SPORT': 'str'},
                 parse_dates=date_column,
                 dayfirst=False)
    dtypes = {
        "DATA_OSS": sqlalchemy.types.DATE(),
        "DESINV": sqlalchemy.types.INTEGER(),
        "DURORI": sqlalchemy.types.INTEGER(),
        "TIPTASSO": sqlalchemy.types.INTEGER(),
        "VALORE": sqlalchemy.types.INTEGER(),
        "CLASSE_ACCORD":sqlalchemy.types.TEXT()
    }
    DataMax =df['DATA_OSS'].max()
    print(f"🔍 ok tabella: {tabella}...DataMax: {DataMax}") 
    df.to_sql(tabella, sqlite, if_exists='replace', index=False, dtype=dtypes)
    existing_tables = [row[0] for row in ddb.execute("SHOW TABLES").fetchall()]
    if tabella in existing_tables:
        ddb.execute(f"DROP TABLE {tabella}") 
    ddb.execute(f"CREATE TABLE {tabella} AS SELECT * FROM df LIMIT 0")  # Crea una tabella vuota con la struttura del DataFrame
    ddb.execute(f"INSERT INTO {tabella} SELECT * FROM df")
    return df

In [11]:
for tabella in STAATER:
    df = carica_dati_in_sql(tabella)

🔍 ok tabella: TDB20207...DataMax: 2024-12-31 00:00:00
🔍 ok tabella: TDB20212...DataMax: 2024-12-31 00:00:00
🔍 ok tabella: TDB20220...DataMax: 2024-12-31 00:00:00
🔍 ok tabella: TDB10222...DataMax: 2024-12-31 00:00:00
🔍 ok tabella: TDB10225...DataMax: 2024-12-31 00:00:00
🔍 ok tabella: TDB10227...DataMax: 2024-12-31 00:00:00
🔍 ok tabella: TDB20210...DataMax: 2024-12-31 00:00:00
🔍 ok tabella: TDB20225...DataMax: 2024-12-31 00:00:00
🔍 ok tabella: TDB20230...DataMax: 2024-12-31 00:00:00
🔍 ok tabella: TDB10195...DataMax: 2024-12-31 00:00:00


In [5]:
len(STAFINRA)

50

## STAATER

- 🔍 ok tabella: TDB20207...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TDB20212...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TDB20220...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TDB10222...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TDB10225...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TDB10227...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TDB20210...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TDB20225...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TDB20230...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TDB10195...DataMax: 2024-12-31 00:00:00

## STACORIS
63 tabelle

- 🔍 ok tabella: TRI30266...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30267...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30265...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30271...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30601...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30602...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30603...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30604...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30605...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30606...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30486...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30496...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30507...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30516...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30524...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30529...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30631...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30632...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30633...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30634...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30635...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30636...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30431...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30446...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30466...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30476...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30361...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30401...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30101...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30126...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30146...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30156...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30190...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30136...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30166...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30871...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30881...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30890...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30900...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30950...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30951...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30952...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI31100...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI31101...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30021...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30171...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30181...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30268...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30269...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30206...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30031...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30231...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30226...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30033...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30211...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30241...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30251...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30290...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30953...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30954...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI30955...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI31102...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TRI31103...DataMax: 2024-12-31 00:00:00


## STASFINRA
50 tabelle

- 🔍 ok tabella: TDB10290...DataMax: 2025-01-31 00:00:00
- 🔍 ok tabella: TDB10221...DataMax: 2025-01-31 00:00:00
- 🔍 ok tabella: TDB10224...DataMax: 2025-01-31 00:00:00
- 🔍 ok tabella: TDB10226...DataMax: 2025-01-31 00:00:00
- 🔍 ok tabella: TDB10295...DataMax: 2025-01-31 00:00:00
- 🔍 ok tabella: TDB20224...DataMax: 2025-01-31 00:00:00
- 🔍 ok tabella: TDB20226...DataMax: 2025-01-31 00:00:00
- 🔍 ok tabella: TDB20290...DataMax: 2025-01-31 00:00:00
- 🔍 ok tabella: TDB20295...DataMax: 2025-01-31 00:00:00
- 🔍 ok tabella: TFR40020...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR20232...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR20255...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR20231...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR30274...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10425...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10435...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR20281...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10254...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR30309...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR30315...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR20163...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR20269...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR20267...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR40082...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR40087...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR40500...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR40300...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR40400...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR30980...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR30970...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10194...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10286...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10241...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10232...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10255...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10281...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10236...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10420...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10430...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10460...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10194...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10283...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10287...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10163...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10269...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10267...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR40100...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10288...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR10289...DataMax: 2024-12-31 00:00:00
- 🔍 ok tabella: TFR20236...DataMax: 2024-12-31 00:00:00

In [5]:
ddb.close()

In [ ]:
query = """SELECT * FROM TDB10295 limit 10;"""
ddb.execute(query).fetchdf() 


In [ ]:
query = """SELECT * FROM STACORIS limit 10;"""
ddb.execute(query).fetchdf() 

## legend, domain, structure

In [ ]:
# merge di file informativi
def merge_csv_files(folder_path, output_file, file):
    all_files = [f for f in os.listdir(folder_path) if f.endswith(".csv") and f.startswith(file)]
    df_list = []
    
    for file_name in all_files:
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path, delimiter=';', dtype=str)  # Legge i CSV come stringhe
        df['File_Origine'] = file_name  # Aggiunge la colonna con il nome del file
        df_list.append(df)
    
    merged_df = pd.concat(df_list, ignore_index=True)
    merged_df.to_csv(output_file, index=False, sep=';')
    print(f"File unito salvato in: {output_file}")

In [17]:
file = 'LEGEND'  # Cambiare

In [ ]:
folder_path = f"D:\\files\\csv\\Bankit"
output_file = f"D:\\files\\csv\\Bankit\\{file}.csv"
merge_csv_files(folder_path, output_file, file)

In [18]:
df = pd.read_csv(file+'.csv', sep =';')
df.to_sql(file, sqlite, if_exists='replace')

1203

In [19]:
df = pd.read_csv('io_tavole.tsv', sep ='\t')
df.to_sql('tabelle', sqlite, if_exists='replace')

89

## **BAM**

In [21]:
tabella = 'MIR0300' # ATECO100,BSIB0600,BSIB0700,BSIB1010,MIR0100
file = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
result = requests.get(file)
date_column = ['DATA_OSS']
data = pd.read_csv(BytesIO(result.content),compression='zip', header=0, sep=';', 
                   quotechar='"', encoding='utf-8',parse_dates=date_column, dayfirst=False)
df = data.melt(id_vars=['DATA_OSS'], var_name='variabile', value_name='valore')
df.to_sql(tabella, sqlite, if_exists='replace')


2640

In [32]:
data.columns = data.columns.str.replace(r'BAM_ATECO.M.1070001.52000700.101.IT.', '')
data.dropna()
melted_df = data.melt(id_vars=['DATA_OSS'], var_name='Elemento', value_name='VALORE')

melted_df[['target','settore']] = melted_df['Elemento'].str.split('.', n=1, expand=True)
melted_df['VALORE'] = melted_df['VALORE']*1000000

In [5]:
dtypes = {"DIVISA1": sqlalchemy.types.INTEGER(), "DURORI": sqlalchemy.types.INTEGER(),"LOC_SPORT": sqlalchemy.types.INTEGER(),
         "VALORE": sqlalchemy.types.INTEGER()}
melted_df[['DATA_OSS', 'target', 'settore', 'VALORE']].to_sql('ATECO100', sqlite, if_exists='replace', dtype=dtypes,index=False)

21372

### BSIB0800 - Prestiti ai residenti in Italia, per durata e tipologia 

### verifica punti di rottura

In [ ]:
SBI33 = data.query('SET_CTP == "SBI33"')[['DATA_OSS','LOC_CTP','VALORE']]
# SBI33['DATA_OSS'] = pd.to_datetime(SBI33['DATA_OSS'])
SBI33.set_index(SBI33['DATA_OSS'], inplace = True)
ts = SBI33['VALORE']
plt.plot(ts, linewidth=3, color='red')
plt.title('')
plt.grid()
plt.show()

In [ ]:
import ruptures as rpt
model = rpt.Dynp(model="l1")  
model.fit(y)
result = model.predict()
rpt.display(SBI33, result)
plt.show()

In [11]:
ddb.execute("DESCRIBE TDB10295").fetchdf()

,column_name,column_type,null,key,default,extra
0,DATA_OSS,TIMESTAMP_NS,YES,None,None,None
1,ENTE_SEGN,VARCHAR,YES,None,None,None
2,FENEC,VARCHAR,YES,None,None,None
3,LOC_CTP,VARCHAR,YES,None,None,None
4,SET_CTP,VARCHAR,YES,None,None,None
5,VALORE,DOUBLE,YES,None,None,None
6,STATUS,DOUBLE,YES,None,None,None


In [ ]:
import duckdb

# Connessione al database DuckDB
ddb = duckdb.connect('D:/Bankit.duckdb')

# Creazione della vista
query = """
CREATE OR REPLACE VIEW V_TDB10295 AS
SELECT 
*
FROM data d
LEFT JOIN stamen s1 ON d.ENTE_SEGN = s1.Elemento
LEFT JOIN stamen s2 ON d.LOC_CTP = s2.Elemento
LEFT JOIN stamen s3 ON d.SET_CTP = s3.Elemento
LEFT JOIN stamen s4 ON d.FENEC = s4.Elemento;
"""

# Esegui la query per creare la vista
ddb.execute(query)

# Verifica che la vista sia stata creata correttamente
print(ddb.execute("DESCRIBE data_enriched").fetchdf())  # Mostra la struttura della vista
